# Extract data from The National Map 3DEP WCS Service

See https://geoscripting-wur.github.io/PythonRaster/

and this code is from https://gist.github.com/rsignell-usgs/3d450d6e3a08f2baf2e5b9e177aa53b1

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from owslib.wcs import WebCoverageService

endpoint = (
    "https://elevation.nationalmap.gov/arcgis/services/3DEPElevation/ImageServer/WCSServer"
    "?request=GetCapabilities"
    "&service=WCS"
)
wcs = WebCoverageService(endpoint, version="1.0.0", timeout=60)

In [3]:
for k, v in wcs.contents.items():
    print(k, v.title)

DEP3Elevation DEP3Elevation
DEP3Elevation_Hillshade Gray DEP3Elevation_Hillshade Gray
DEP3Elevation_Aspect Degrees DEP3Elevation_Aspect Degrees
DEP3Elevation_Aspect Map DEP3Elevation_Aspect Map
DEP3Elevation_Contour 25 DEP3Elevation_Contour 25
DEP3Elevation_Hillshade Elevation Tinted DEP3Elevation_Hillshade Elevation Tinted
DEP3Elevation_Height Ellipsoidal DEP3Elevation_Height Ellipsoidal
DEP3Elevation_GreyHillshade_elevationFill DEP3Elevation_GreyHillshade_elevationFill
DEP3Elevation_Hillshade Multidirectional DEP3Elevation_Hillshade Multidirectional
DEP3Elevation_Slope Map DEP3Elevation_Slope Map
DEP3Elevation_Slope Degrees DEP3Elevation_Slope Degrees
DEP3Elevation_Contour Smoothed 25 DEP3Elevation_Contour Smoothed 25


In [4]:
v = wcs["DEP3Elevation"]
print(v.title)
print(v.boundingBoxWGS84)
print(v.supportedFormats)

DEP3Elevation
(-179.99998854118687, -15.001663244822502, 179.99999272129153, 84.00167857213272)
['GeoTIFF', 'HDF']


In [30]:
# boulder
boulder = (-105.2705, 40.0150)
delta = 0.5
center = boulder
bbox = (center[0] - delta, center[1] - delta, center[0] + delta, center[1] + delta,)
res = 0.0003
output = wcs.getCoverage(
    identifier="DEP3Elevation",
    bbox=bbox,
    crs="EPSG:4326",
    format="GeoTIFF",
    resx=res,
    resy=res
)

In [31]:
with open('boulder.tif', 'wb') as file:
    file.write(output.read())

In [32]:
import xarray as xr
import hvplot.xarray
from io import BytesIO
from rasterio.io import MemoryFile

#sio = BytesIO(output.read())  # Create an in-memory stream of the content


da = xr.open_rasterio('boulder.tif')[0].drop_vars("band")

In [33]:
da.shape

(3333, 3333)

In [34]:
da.hvplot.image(x="x", y="y", geo=True, rasterize=True, cmap="rainbow")

:DynamicMap   []
   :Image   [x,y]   (value)

In [35]:
from xrspatial import hillshade
da_hs = hillshade(da)

In [36]:
da_hs.hvplot.image(rasterize=True, geo=True, colormap="gray")

:DynamicMap   []
   :Image   [x,y]   (hillshade)